In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv('./data/20210514/AMD.csv', sep=';')
data['date'] = pd.to_datetime(data['date'])
quotes = data.copy()

In [5]:
quotes['ask'] = np.where(quotes['ask'].values > 0.0, quotes['ask'].values, np.nan)
quotes['bid'] = np.where(quotes['bid'].values > 0.0, quotes['bid'].values, np.nan)
quotes['trade'] = np.where(quotes['trade'].values > 0.0, quotes['trade'].values, np.nan)

In [6]:
quotes['trade'] = quotes['trade'].fillna(method='ffill')

In [7]:
# quotes[['ask', 'bid', 'trade']].plot(figsize=(20, 10))
# ax = quotes['trade'].plot(figsize=(20, 10), legend=True)
# quotes[['ask', 'bid']].plot(secondary_y=True, ax=ax, legend=True)


In [21]:
trades = quotes['trade']

asks = quotes['ask'].fillna(method='ffill')
asks_change = (asks.shift(1) / asks - 1.0)
asks_change_std = asks_change.std()
asks_change_mean = asks_change.mean()
asks_zscore = (np.abs(asks_change) - np.abs(asks_change_mean)) / asks_change_std
sell = np.where(asks_zscore > 3.5, quotes['bid'].fillna(method='bfill'), np.nan)

sma_long = trades.rolling(100).mean()
sma_short = trades.rolling(50).mean()
buy = np.where((sma_long < sma_short) & (sma_long.shift(1) > sma_short.shift(1)), quotes['ask'].fillna(method='bfill'), np.nan)

quotes['sell'] = sell
quotes['buy'] = buy
quotes['sma_long'] = sma_long
quotes['sma_short'] = sma_short

buy_price = 0.0
position = 0
capital = 100_000.0
order_fee = 1.0
transaction_fee_ratio = 0.0
for index, row in quotes.iterrows():
    if position == 0 and row['buy'] > 0.0:
        buy_price = row['buy']
        position = int(capital / buy_price)
        transaction = position * buy_price
        transaction *= 1.0 + (transaction_fee_ratio / 100.0)
        transaction += order_fee
        capital -= transaction
        print(f'buy:  $ {transaction:.2f}')
    elif position > 0 and row['sell'] > 0.0:
        sell_price = row['sell']
        buy_transaction = position * buy_price
        sell_transaction = position * sell_price
        sell_transaction -= sell_transaction * (transaction_fee_ratio / 100.0)
        brutto = sell_transaction - buy_transaction
        tax = 0
        if brutto > 0.0:
            tax = 0.25 * 1.055 * brutto
        capital += buy_transaction + brutto - tax
        capital -= order_fee
        print(f'sell: $ {sell_transaction:.2f}')
        buy_price = 0.0
        position = 0
print(f'Total: $ {capital + (position * buy_price):.2f}')

buy:  $ 99974.17
sell: $ 99445.50
buy:  $ 99400.04
sell: $ 99385.52
buy:  $ 99386.52
sell: $ 99412.56
buy:  $ 99454.12
sell: $ 99480.16
buy:  $ 99475.13
sell: $ 99447.11
buy:  $ 99422.30
sell: $ 99394.32
buy:  $ 99375.56
sell: $ 99361.08
buy:  $ 99375.56
sell: $ 99388.04
buy:  $ 99375.56
sell: $ 99873.32
buy:  $ 99786.53
sell: $ 100487.01
buy:  $ 100237.62
sell: $ 100236.62
buy:  $ 100257.30
sell: $ 100242.85
buy:  $ 100257.30
sell: $ 100256.30
buy:  $ 100275.24
sell: $ 100099.78
buy:  $ 100035.34
sell: $ 99860.53
buy:  $ 99922.84
sell: $ 100122.54
buy:  $ 100048.71
sell: $ 100074.45
buy:  $ 100059.25
sell: $ 100044.90
buy:  $ 100059.25
sell: $ 100084.95
buy:  $ 100037.66
sell: $ 100023.32
buy:  $ 100072.60
sell: $ 100125.00
buy:  $ 100064.34
sell: $ 100063.34
buy:  $ 100077.68
sell: $ 99543.08
buy:  $ 99543.24
sell: $ 98160.44
buy:  $ 98126.30
sell: $ 98099.00
buy:  $ 98165.75
sell: $ 98151.60
buy:  $ 98117.38
sell: $ 98103.24
buy:  $ 98080.14
sell: $ 96846.99
buy:  $ 96860.96
sell: $

In [ ]:
ax = quotes[['trade']].plot(figsize=(20, 10))
quotes[['sell', 'buy']].plot(ax=ax, marker='o')

In [54]:
quotes[100:][['prev_sma_short', 'sma_short']]


,prev_sma_short,sma_short
100,5.900,5.900
101,5.900,5.900
102,5.900,5.900
103,5.900,5.900
104,5.900,5.900
...,...,...
5499,6.160,6.160
5500,6.160,6.160
5501,6.160,6.159
5502,6.159,6.158
